In [1]:
# calculation is wrong - the resulting index should be between 0 and 1
#I am reading the top 10 generality values here and dissecting them

# the matching in generality.ipynb is generating many NAN
# this should not happen 
# - wipo is the list of classes each patent is classified into
# - patentcitation is the list of citations

# if you merge wipo into patentcitation, there always be a classification for a cited patent
# some level of error is acceptable, but the number of missing values is currently at 400k

# this script matches and compares both dataset
# the goal is to understand why this is happening

# i can match with merge, join, concat and append
# i can also match with cleanuspatentcitation vs uspatentcitation
# also wipo vs wipo_horiz


In [2]:
import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler

import pandas as pd
import numpy as np
import re

In [3]:
generality = '/home/rkogeyam/PATENT_CITATION/data/generality.csv'

df=pd.read_csv(generality)

In [4]:
df.head()

,citation_id,df_squared,total_citation,herfindal,output,field_id_1,field_id_10,field_id_11,field_id_12,field_id_13,...,field_id_32,field_id_33,field_id_34,field_id_35,field_id_4,field_id_5,field_id_6,field_id_7,field_id_8,field_id_9
0,0,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000004,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000006,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000482,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.tail()

,citation_id,df_squared,total_citation,herfindal,output,field_id_1,field_id_10,field_id_11,field_id_12,field_id_13,...,field_id_32,field_id_33,field_id_34,field_id_35,field_id_4,field_id_5,field_id_6,field_id_7,field_id_8,field_id_9
5,0000H94,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0001552,27.0,9,3.0,-2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,000169,18.0,9,2.0,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,000280,2.0,1,2.0,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,000312,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,df_squared,total_citation,herfindal,output,field_id_1,field_id_10,field_id_11,field_id_12,field_id_13,field_id_14,...,field_id_32,field_id_33,field_id_34,field_id_35,field_id_4,field_id_5,field_id_6,field_id_7,field_id_8,field_id_9
count,10.000000,10.000000,10.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,5.400000,2.600000,1.400000,-0.400000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9.264028,3.373096,0.699206,0.699206,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,1.000000,1.000000,-2.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.000000,1.000000,1.000000,-0.750000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.750000,1.000000,1.750000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,27.000000,9.000000,3.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
